In [43]:
import json
import pandas as pd
from pandas import json_normalize
#from pandas.json_normalize import json_normalize

import os
import configparser
import datetime as dt
from datetime import datetime
from os.path import exists

# Method to read config file settings
def read_config(Config_File):
    config = configparser.ConfigParser()
    config.read(Config_File)
    return config

configurations = read_config("configurations.ini")

In [4]:
for each_section in configurations.sections():
    print("Section: {}".format(each_section))
    for (each_key, each_val) in configurations.items(each_section):
        print("key {} : Value {}".format(each_key,each_val))

Section: common_settings
key gtfs_history_data_root : Value C:\Users\AYAB\OneDrive - Drexel University\drexel\DSCI 591\detour project\data\GTFS-RT-historical-2022


In [5]:
print(configurations['common_settings']['gtfs_history_data_root'])

C:\Users\AYAB\OneDrive - Drexel University\drexel\DSCI 591\detour project\data\GTFS-RT-historical-2022


In [95]:
#load 1 file in panda frame
month = '10'
day = '3'
hour = '14'
minute = '16'

file = configurations['common_settings']['gtfs_history_data_root'] + "\\" + month + '\\' + day + '\\' + hour + '\\' + minute + '\\' + 'feed.json'

with open(file) as json_file:
    data = json.load(json_file)
    #print(data['entity'])

# gtfs_rt_entities = json_normalize(data,errors='ignore', record_path=["entity"]
#                                                 ,meta=[["id"],
#                                                      ["vehicle","trip"]] )
# load json into flat dictionary
gtfs_rt_entities = json_normalize(data['entity'])
gtfs_rt_entities['file'] = "test"
# initial data cleaning, remove the vehicle that have not trip/route data
#gtfs_rt_entities = gtfs_rt_entities.dropna(subset = ['vehicle.trip.tripId'])
#gtfs_rt_entities.head(5)

#gtfs_rt_entities.to_csv('data\out.csv',index=False)

gtfs_rt_entities.head(5)

,id,vehicle.trip.tripId,vehicle.trip.startDate,vehicle.trip.routeId,vehicle.position.latitude,vehicle.position.longitude,vehicle.position.bearing,vehicle.position.speed,vehicle.currentStopSequence,vehicle.currentStatus,vehicle.timestamp,vehicle.stopId,vehicle.vehicle.id,vehicle.vehicle.label,file
0,8099,48492,20221003,130,40.239307,-74.96527,121.643326,0.013411,1.0,STOPPED_AT,1664806564,29691,8099,8099,test
1,8093,NaN,NaN,NaN,40.113270,-75.34523,NaN,0.008941,NaN,NaN,1664806565,NaN,8093,8093,test
2,8094,51953,20221003,99,40.113370,-75.34452,116.468094,0.017882,1.0,STOPPED_AT,1664806565,1649,8094,8094,test
3,8092,89875,20221003,15,39.970444,-75.14830,93.077510,0.013411,37.0,STOPPED_AT,1664806566,21091,8092,8092,test
4,8097,49425,20221003,139,40.137850,-75.51939,80.286340,7.031939,31.0,STOPPED_AT,1664806566,28948,8097,8097,test


In [107]:
#TODO: create a parameter for file output

from numpy import NaN


def load_gtfs_rt(start_date_time_str, stop_date_time_str):
    """ load GTFS RT file into a panda DF and store result to csv file"""
    start_date_time = datetime.strptime(start_date_time_str, '%m/%d/%Y %H:%M')
    stop_date_time = datetime.strptime(stop_date_time_str, '%m/%d/%Y %H:%M')
    
    # delta time
    delta = dt.timedelta(minutes=1)
    
    #initialize the file with header for dataframe
    with open('data\out.csv', 'w') as f:
        f.write('file,id,in_service,vehicle.currentStatus,vehicle.currentStopSequence,vehicle.position.bearing, \
            vehicle.position.latitude,vehicle.position.longitude,vehicle.position.speed,vehicle.stopId, \
            vehicle.timestamp,vehicle.trip.routeId,vehicle.trip.startDate,vehicle.trip.tripId,vehicle.vehicle.id,vehicle.vehicle.label')
        f.write("\n") # need to add the carriage return this way

    # iterate over range of date / time
    while (start_date_time <= stop_date_time):
        # print(start_date_time, end="\n")
        #generate the file name to open
        file = configurations['common_settings']['gtfs_history_data_root'] + "\\" \
            + start_date_time.strftime("%m") + '\\' \
            + start_date_time.strftime("%#d") + '\\' \
            + start_date_time.strftime("%H") + '\\' \
            + start_date_time.strftime("%M") + '\\' + 'feed.json'

        #load the file name into mem
        #TODO: if I don't find a file fail gracefully. print out the file not found.

        if exists(file):
            with open(file) as json_file:
                data = json.load(json_file)

            # load json into flat dictionary

            if "entity" in data:
                gtfs_rt_entities = json_normalize(data['entity'])
                gtfs_rt_entities['file'] = start_date_time # add the file date/time in the output
                gtfs_rt_entities['in_service'] = [ 1 if val == '' else 0 for val in gtfs_rt_entities['vehicle.trip.tripId']] # check if the vehicle is in service or not
                gtfs_rt_entities.sort_index(axis=1, inplace=True)

            else:
                print("entiry is key in not present in file {}".format(start_date_time))
            # initial data cleaning, remove the vehicle that have not trip/route data
            #gtfs_rt_entities = gtfs_rt_entities.dropna(subset = ['vehicle.trip.tripId'])

            # append the flatten data to csv file
            gtfs_rt_entities.to_csv('data\out.csv', mode='a', index=False, header=True)
        else:
            # if file is missing print a log
            print('file date/time not available: {}\\feed.json '.format(start_date_time.strftime("%m/%d/%Y %H:%M")))
        # increemnt datetime by 1 minute
        start_date_time += delta


In [108]:
# start_Date_time_str = "10/3/2022 9:00"

# start_Date_time = datetime.strptime(start_Date_time_str, '%m/%d/%Y %H:%M')

load_gtfs_rt("10/03/2022 9:00","10/03/2022 19:59")

# print('shape {}'.format(gtfs_rt_entities.shape))

file date/time not available: 10/03/2022 13:54\feed.json 
file date/time not available: 10/03/2022 18:47\feed.json 
file date/time not available: 10/03/2022 18:49\feed.json 
file date/time not available: 10/03/2022 18:50\feed.json 
file date/time not available: 10/03/2022 18:51\feed.json 
entiry is key in not present in file 2022-10-03 18:52:00
file date/time not available: 10/03/2022 18:53\feed.json 
file date/time not available: 10/03/2022 18:54\feed.json 
file date/time not available: 10/03/2022 18:55\feed.json 
file date/time not available: 10/03/2022 18:56\feed.json 
file date/time not available: 10/03/2022 18:58\feed.json 
file date/time not available: 10/03/2022 18:59\feed.json 
file date/time not available: 10/03/2022 19:01\feed.json 
file date/time not available: 10/03/2022 19:03\feed.json 
file date/time not available: 10/03/2022 19:05\feed.json 
file date/time not available: 10/03/2022 19:06\feed.json 
file date/time not available: 10/03/2022 19:07\feed.json 
file date/time 

In [ ]:
df = pd.read_csv('data\out.csv')

df.head(5)